# VacationPy

---


In [5]:
# dependencies and setup
import hvplot.pandas
import matplotlib.pyplot as plt
import requests
import pandas as pd
from api_keys import geoapify_key

In [7]:
city_df = pd.read_csv('output_data/cities.csv')
city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


## step 1: create a map for every city in the data frame

In [66]:
map_plot_1 = city_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 1,
    color = 'City'
)

map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

## Step 2: narrow down City Data to find my ideal weather conditions

In [37]:
# trim dataset to conditions i want to go to

cond_1 = (city_df['Max Temp'] < 30) & (city_df['Max Temp'] >= 20)
cond_2 = (df_1['Humidity'] < 60)
cond_3 = (df_2['Wind Speed'] < 20)
cond_4 = (df_3['Cloudiness'] <= 15)

df_1 = city_df.loc[cond_1,:]
df_2 = df_1.loc[cond_2,:]
df_3 = df_2.loc[cond_3,:]
destinations_df = df_3.loc[cond_4,:]
destinations_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
73,73,nalut,30.3333,10.8500,25.41,27,0,5.11,LY,1666108275
239,239,rawson,-43.3002,-65.1023,23.74,25,2,13.52,AR,1666108102
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
258,258,comodoro rivadavia,-45.8667,-67.5000,25.92,24,0,7.20,AR,1666108389
305,305,kingman,35.1894,-114.0530,20.07,49,5,4.02,US,1666108417
317,317,salalah,17.0151,54.0924,25.49,59,7,1.67,OM,1666108424
340,340,santa rosalia,27.3167,-112.2833,24.62,56,0,0.74,MX,1666108436
363,363,narwar,25.6500,77.9000,22.35,55,0,1.29,IN,1666108449
391,391,saint-francois,46.4154,3.9054,23.69,57,0,4.12,FR,1666108465


## step 3: create a new Data Frame called Hotel_df

In [47]:
# create new dafaframe to store valuable series and add the hotel name series
hotel_df = destinations_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
hotel_df['Hotel Name']= ''
hotel_df


/var/folders/58/wf3rb_4174q5hgcffz8xp_640000gn/T/ipykernel_50008/2272076868.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']= ''


,City,Country,Lat,Lng,Humidity,Hotel Name
63,banda,IN,25.4833,80.3333,52,
73,nalut,LY,30.3333,10.8500,27,
239,rawson,AR,-43.3002,-65.1023,25,
240,tikaitnagar,IN,26.9500,81.5833,59,
258,comodoro rivadavia,AR,-45.8667,-67.5000,24,
305,kingman,US,35.1894,-114.0530,49,
317,salalah,OM,17.0151,54.0924,59,
340,santa rosalia,MX,27.3167,-112.2833,56,
363,narwar,IN,25.6500,77.9000,55,
391,saint-francois,FR,46.4154,3.9054,57,


## For each city, use geoapify API to find the first hotel located within 10,000 metres of the coordinates

In [61]:
# set up parameters for hotel search:
radius = 10000
params = {
    'categories':'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 20
}

# 
print('Starting Hotel Search')

#iterate
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng'] 
    params['filter'] = f"circle:{longitude},{latitude},{radius}"
    params['bias'] = f"proximity:{longitude},{latitude}"
    base_url = "https://api.geoapify.com/v2/places"

    name_adress = requests.get(base_url, params=params)
    name_adress = name_adress.json()

    #grab the first hotel
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_adress['features'][0]['properties']['name']
    except (KeyError, IndexError):
        #if no hotel is found set the name as no hotel
        hotel_df.loc[index, 'Hotel Name'] = 'No hotel found'

    #log search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

hotel_df

Starting Hotel Search
banda - nearest hotel: #acnindiafy21
nalut - nearest hotel: No hotel found
rawson - nearest hotel: Hotel Deportivo
tikaitnagar - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
kingman - nearest hotel: Home2 Suites by Hilton Kingman
salalah - nearest hotel: Muscat International Hotel
santa rosalia - nearest hotel: Sol y Mar
narwar - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
lakki marwat - nearest hotel: No hotel found
tikrit - nearest hotel: فندق بلازا
lahij - nearest hotel: No hotel found
pali - nearest hotel: Hotel Pawan International


,City,Country,Lat,Lng,Humidity,Hotel Name
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
73,nalut,LY,30.3333,10.8500,27,No hotel found
239,rawson,AR,-43.3002,-65.1023,25,Hotel Deportivo
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
258,comodoro rivadavia,AR,-45.8667,-67.5000,24,No hotel found
305,kingman,US,35.1894,-114.0530,49,Home2 Suites by Hilton Kingman
317,salalah,OM,17.0151,54.0924,59,Muscat International Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Sol y Mar
363,narwar,IN,25.6500,77.9000,55,No hotel found
391,saint-francois,FR,46.4154,3.9054,57,Chez Lily
